# Daily Task to Update the Tax Parcels Feature Class


 <b> TO DO: </b>
 <ul>         
        <li>Do not use arc.env.workspace Environment Variables.</li>
        <li>Truncate and Append Tax Exempt to PROD.gisadmin.TaxExempt.</li>
        <li>Truncate and Append Tax Parcels to PROD.gisadmin.TaxParcels.</li>
        <li>Truncate and Append Historical Parcels to PROD.gisadmin.TaxParcels.</li>
        
        
         </ul> 

Import modules

In [ ]:
import pandas as pd
import arcpy as arc
from arcpy import env
from arcgis.gis import GIS
from arcgis import features
import numpy as np
from datetime import datetime as dt
from pathlib import Path

### Create Folders in the Users Directory since it should have r/w permissions for the user.

In [ ]:
arc.env.overwriteOutput = True
arc.env.qualifiedFieldNames = False
arc.env.outputZFlag = 'Disabled' #To remove z data from parcel fabric due to it being a polygonZ
arc.env.outputMFlag = 'Disabled'
now = dt.today()
mStr = now.strftime('%m%Y')
dStr = now.strftime('%m_%d')
wStr = now.strftime('%U')
uPath = Path.home()
locFolders = ['Processing', 'Review']
if uPath.exists():
    for x in locFolders:
        a = Path(uPath / 'GIS' / x)
        if a.exists():
            print(f'{a} already exists.')
        else:
            a.mkdir(parents=True)
            print(f'{a} has been created.')
else:
    pass

gisPath = uPath / 'GIS'
lPath = [f for f in gisPath.glob('*')]
netDir = Path(r'\\kcdp-1\KCGIS\MasterGISFiles\Ben')
netDB = netDir / 'GISPro' / 'SDE Connections'

### Only use with notebook

In [ ]:
pd.options.display.max_columns = 40

### Create File GeoDatabase and Feature Datasets

In [ ]:
#Create Folders for Permits Data
paFolder = [f for f in lPath if f.name == 'Processing'][0]
paProcessing = paFolder / 'Parcels' / f'{dStr}'
if paProcessing.exists() == True:
    print(f'{paProcessing} already exist.')
else:
    paProcessing.mkdir(parents=True)
    print(f'Created {paProcessing}.')

paFR = [f for f in lPath if f.name == 'Review'][0]
paReview = paFR / 'Parcels' / f'{dStr}'
if paReview.exists() == True:
    print(f'{paReview} already exist')
else:
    paReview.mkdir(parents=True)
    print(f'Created {paReview}')

In [ ]:
iE = netDB / 'MAPPINGADMIN.sde' / 'PROD.MAPPINGADMIN.ParcelEditing'
sr = arc.Describe(f'{iE}').spatialReference
outGDB = gisPath / paFolder / f'Data_{mStr}.gdb'
# dsA = gisPath / pFolder / f'{outGDB}.gdb'
locGDB = outGDB / f'Daily_{dStr}'
if arc.Exists(f'{outGDB}'):
    print("GDB already exists.")
else:
    arc.CreateFileGDB_management(f'{paFolder}', f'{outGDB.name}')
    print(f'Created File GeoDatabase at {outGDB.parent}')

time.sleep(2)

if arc.Exists(f'{locGDB}'):
    print(f'{locGDB.name} already exists')
else:
    arc.CreateFeatureDataset_management(f'{locGDB.parent}', f'{locGDB.name}', sr)
    print(f'{locGDB.name} Dataset has been created')

### Does not work with Parcel Fabric due to bug with 10.5.1

In [ ]:
dbOwnTab = 'PROD.gisadmin.OWNERSHIPINFORMATION'

In [ ]:
#create paths to the data instead of using env.workspace makes it so that I can use arc Walk
arc.env.workspace = f'{iE.parent}'
tList = dbOwnTab
cParcels = []
cTables = []
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in tList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}' #Tables need to have a date string since they won't be under the dataset
            # c = locGDB.parent
            if arc.Exists(f'{outGDB / b}'):
                cTables.append(outGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.TableToTable_conversion(f, f'{outGDB}', f'{b}')
                cTables.append(outGDB.joinpath(b))
                print (f'{b} has been copied')

### Works with Parcel Fabric

In [ ]:
#currently using env.workspace to move parcelfabric needs to be changed to use path
pfParcels = 'PROD.MAPPINGADMIN.ParcelFabric_Parcels'
dbPFPlan = 'PROD.MAPPINGADMIN.ParcelFabric_Plans'
exp = "TYPE = 7 AND Historical = 0"
a = pfParcels.split('.')[-1]
b = f'{a}_{dStr}'
# print(f'{iEPath[1]}')
if arc.Exists(f'{locGDB / b}'):
    cParcels.append(locGDB.joinpath(b))
    print(f'{b} already exists')
else:
    print(f'Copying Parcel Fabric to {locGDB.name}')
    #print(a)
    # g = f'{f}'
    #print(g)
    arc.FeatureClassToFeatureClass_conversion(pfParcels, f'{locGDB}', b, where_clause=exp)
    cParcels.append(locGDB.joinpath(b))
    print(f'Finished Copying {b} to {locGDB.name}')
#Copy parcel Fabric plans table to local GDB
c = dbPFPlan.split('.')[-1]
d = f'{c}_{dStr}'
if arc.Exists(f'{outGDB / d}'):
    cTables.append(outGDB.joinpath(d))
    print(f'{d} already exists')
else:
    print(f'Copying {d} to {outGDB.name}')
    arc.TableToTable_conversion(dbPFPlan, f'{outGDB}', d)
    cTables.append(outGDB.joinpath(d))
    print(f'Finised Copying {d} to {outGDB.name}')

In [ ]:
#Create DataFarame from the current Parcels
pfFC = [f for f in cParcels if f.name.startswith('ParcelFabric') == True][0]
pf_df = features.GeoAccessor.from_featureclass(pfFC, fields=['Name', 'Pride', 'Lot_Number', 'Plat_Book','Parent'])
pf_df.Name = pf_df.Name.apply(lambda x: x.strip())
print('Parcel Fabric DataFrame created.')

In [ ]:
#Remove the 1 values from Parcel Fabric
pf_df = pf_df[pf_df.Name != '1'].copy(deep=True)

Use to Create the Historical Parcels Feature Class on esridb

In [ ]:
#currently using env.workspace to move parcelfabric needs to be changed to use path
exp = "TYPE = 7 AND Historical = 1"
ah = pfParcels.split('.')[-1]
bh = f'H{a}_{dStr}'
if arc.Exists(f'{locGDB / bh}'):
    cParcels.append(locGDB.joinpath(b))
    print(f'{bh} already exists')
else:
    print(f'Copying {bh} to {locGDB.name}')
    arc.FeatureClassToFeatureClass_conversion(pfParcels, f'{locGDB}', bh, where_clause=exp)
    cParcels.append(locGDB.joinpath(bh))
    print(f'Finished Copying {bh} to {locGDB.name}')

In [ ]:
#Create DataFrame from the historical Parcels
hpfFC = [f for f in cParcels if f.name.startswith('HParcelFabric') == True][0]
hdf = features.GeoAccessor.from_featureclass(hpfFC, fields=['Name', 'HistoryType', 'Lot_Number', 'Plat_Book', 'Parent'])
hdf.Name = hdf.Name.apply(lambda x: x.strip()) #Remove whitespace
print('Historic Parcel Fabric DataFrame created.')

In [ ]:
#Remove the 1 values from Name column
ehdf = hdf[hdf.Name != '1'].copy(deep=True)

In [ ]:
#Append the new data to esridb feature class
dbOut = netDB / "PROD-GISADMIN.sde" / 'PROD.GISADMIN.Parcel_Information' / 'PROD.GISADMIN.HistoricParcel'
hOut = locGDB / f'HParcels_{dStr}'
print(f'Exporting {hOut.name} to {locGDB.name}')
ehdf.spatial.to_featureclass(str(hOut), sanitize_columns=False)
print('Finished Exporting')
print('Truncating and Appending to HParcels on esridb')
arc.TruncateTable_management(f'{dbOut}')
arc.Append_management(f'{hOut}', f'{dbOut}', schema_type='NO_TEST')
print("Finished adding new data to esridb")
del hdf #This deletes the historic parcels dataframe so it will clear up RAM

Prepare Ownership Table for join with Parcel Fabric

In [ ]:
oTab = [f for f in cTables if f.name.startswith('OWNER') == True][0]
ot = features.GeoAccessor.from_table(f'{oTab}', fields=['PARCELID','LOCATION', 'OWNERNAME', 'SECONDARYOWNER', 'MAILINGADDRESS','MAILINGADDRESS2','OWNERCITY','OWNERSTATE','OWNERZIP','DEEDREFERENCE','DEEDACREAGE','LANDASSESSMENT','IMPROVE','TOTALASSESSMENT','LOCATIONID','YEARBUILT','PROPERTYUSE'])

In [ ]:
oMerge = pf_df.merge(ot, how='outer', right_on='PARCELID', left_on='Name', indicator=True)

In [ ]:
#Export Parcels that don't match
nM = paReview / 'Parcels_Missing.csv' 
oMerge[(oMerge._merge == 'right_only') & (oMerge.PARCELID.str.contains('-00001', na=False))].to_csv(nM, index=False) #Export parcels with -00001 that don't match, excludes different suffixes

In [ ]:
oMerge.reset_index(drop=True, inplace=True)
oMerge = oMerge[(oMerge._merge == 'both') & (oMerge.SHAPE.notnull())].copy(deep=True)

In [ ]:
#Parcels that should possibly be combined
combPar = paReview / 'MergeParcels.csv'
oMerge[oMerge.Name.duplicated() == True].to_csv(combPar, index=False)
oMerge.drop(columns='_merge', inplace=True)

Prepare Parcel Fabric Plans Table for join with Ownership and Parcel Fabric (oMerge)

In [ ]:
pfTab = [f for f in cTables if f.name.startswith('ParcelFabric') == True][0]
pft = features.GeoAccessor.from_table(f'{pfTab}', fields=['Name','Description', 'Surveyor', 'Company', 'SurveyDate', 'LegalDate', 'ModifyDate'])

In [ ]:
#Drop Duplicates from Parcel Fabric Plans
parDups = paReview / 'Plans_Dups.csv'
pft[pft.Name.duplicated(keep=False)].to_csv(parDups, index=False)

pft.sort_values(by='ModifyDate', inplace=True)
pft.drop_duplicates(subset=['Name'], keep='last', inplace=True)
pft.drop(columns='ModifyDate', inplace=True)
pft.reset_index(drop=True, inplace=True)

In [ ]:
pfOMerge = oMerge.merge(pft, how='outer', left_on='Plat_Book', right_on='Name', indicator=True)
bPf = pfOMerge[pfOMerge._merge != 'right_only'].copy(deep=True)

In [ ]:
#Export Plat Books that don't match or missing
planNM = paReview / 'Missing_Plans.csv'
pfOMerge[pfOMerge._merge == 'right_only'].to_csv(planNM, index=False)

In [ ]:
#Change the data types of the columns to match the values that it contains
ftype = {'TOTALASSESSMENT' : 'int32', 'DEEDACREAGE': 'int32', 'LANDASSESSMENT': 'int32', 'IMPROVE': 'int32', 'LOCATIONID': 'int32', 'YEARBUILT': 'int32'}
bPf = bPf[bPf.columns.tolist()].astype(ftype)

#drop the additional Name column and rename the other Name column
bPf.drop(columns=['Name_y', '_merge'], inplace=True)
bPf.rename(columns={'Name_x': 'Name'}, inplace=True)
bPf.reset_index(drop=True, inplace=True)

#Fill in the null values
bPf.fillna('', inplace=True)

In [ ]:
#export to local gdb
curPar = locGDB / f'Parcels_{dStr}'
bPf.spatial.to_featureclass(curPar, sanitize_columns=False)
print(f'Finised Exporting {curPar.name} to {locGDB.name}')

#Copy to esridb
pdbOut = netDB / "PROD-GISADMIN.sde" / 'PROD.GISADMIN.Parcel_Information' / 'PROD.GISADMIN.TaxParcel'
# arc.FeatureClassToFeatureClass_conversion(str(curPar), f'{pdbOut.parent}', f'{pdbOut.name}')
print(f'Truncating and Appending to {pdbOut.name} on esridb')
arc.TruncateTable_management(f'{pdbOut}')
arc.Append_management(f'{curPar}', f'{pdbOut}', schema_type='NO_TEST')
print("Finished adding new data to esridb")